In [ ]:
import sys
import warnings
if not sys.warnoptions:
  warnings.simplefilter("ignore")

In [ ]:
! pip install deap
! pip install scoop

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import sklearn.model_selection as model_selection
import pickle, scipy, csv, statistics, math, warnings, joblib, random, numpy
from sklearn import metrics, svm, datasets
from math import log,e, floor
from time import time
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score,cross_val_predict,GridSearchCV, train_test_split, StratifiedKFold, KFold, cross_validate, learning_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import plot_confusion_matrix, mean_absolute_error, accuracy_score, r2_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score
from sklearn import metrics, preprocessing, svm
from sklearn.linear_model import LogisticRegression
from deap import creator, base, tools, algorithms
from scoop import futures
from sklearn.utils import shuffle
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def emotion_label(labels, class_label):
	em_labels = []
	if(class_label == "valence"):
		for i in range(0, labels.shape[0]):
			if (labels[i][0]>5): # high valence
				em_labels.append(1)
			else: # low valence
				em_labels.append(0)
		return em_labels
	elif(class_label == "arousal"):
		for i in range(0, labels.shape[0]):
			if (labels[i][1]>5): # high arousal
				em_labels.append(1)
			else: # low arousal
				em_labels.append(0)
		return em_labels
	elif(class_label == "all"):
		for i in range(0, labels.shape[0]):
			if (labels[i][0]>5): # high valence
				if(labels[i][1]>5): # high arousal
					em_labels.append(1) # HVHA
				else:
					em_labels.append(0) # HVLA
			else: # low valence
				if(labels[i][1]>5): # high arousal
					em_labels.append(2) # LVHA
				else: # low arousal
					em_labels.append(3) # LVLA
		return em_labels

In [ ]:
def kfold(allFeatures, allClasses, index):
    m = int(allClasses.shape[0])
    s = int(m/10)
    train_index = np.ones(m-s+1).astype(int)
    test_index  = np.ones(s).astype(int)
    for i in range(s*index,s*(index+1)):
        test_index[i-s*index] = i
    for i in range(0,s*index):
        train_index[i] = i
    for i in range(s + s*index,m):
        train_index[i-s] = i
    X_train = allFeatures.iloc[train_index]
    X_test = allFeatures.iloc[test_index]
    y_train = allClasses[train_index]
    y_test = allClasses[test_index]
    # normalize the xtrain data only with class labels
    return X_train, X_test, y_train, y_test

In [ ]:
# Feature subset fitness function
def getFitness(individual, X_train, X_test, y_train, y_test):
    """
    parse our feature columns that we don't use
    apply one hot encoding to the features.
    """
    cols = [index for index in range(len(individual)) if individual[index] == 0]
    X_trainParsed = X_train.drop(X_train.columns[cols], axis=1)
    X_trainOhFeatures = pd.get_dummies(X_trainParsed)
    X_testParsed = X_test.drop(X_test.columns[cols], axis=1)
    X_testOhFeatures = pd.get_dummies(X_testParsed)

    # Remove any columns that aren't in both the training and test sets
    sharedFeatures = set(X_trainOhFeatures.columns) & set(X_testOhFeatures.columns)
    removeFromTrain = set(X_trainOhFeatures.columns) - sharedFeatures
    removeFromTest = set(X_testOhFeatures.columns) - sharedFeatures
    X_trainOhFeatures = X_trainOhFeatures.drop(list(removeFromTrain), axis=1)
    X_testOhFeatures = X_testOhFeatures.drop(list(removeFromTest), axis=1)

    # Apply logistic regression on the data, and calculate accuracy
    clf = svm.SVC(kernel='poly').fit(X_trainOhFeatures, y_train)
    predictions = clf.predict(X_testOhFeatures)
    accuracy = accuracy_score(y_test, predictions)*100
    return (accuracy,)

In [ ]:
def getFitness_all(individual, X_train, X_test, y_train, y_test):
    """
    Feature subset fitness function
    Parse our feature columns that we don't use
    Apply one hot encoding to the features.
    """
    cols = [index for index in range(len(individual)) if individual[index] == 0]
    #print(cols)
    #print(cols.shape)
    X_trainParsed = X_train.drop(X_train.columns[cols], axis=1)
    X_trainOhFeatures = pd.get_dummies(X_trainParsed)
    X_testParsed = X_test.drop(X_test.columns[cols], axis=1)
    X_testOhFeatures = pd.get_dummies(X_testParsed)

    # Remove any columns that aren't in both the training and test sets
    sharedFeatures = set(X_trainOhFeatures.columns) & set(X_testOhFeatures.columns)
    removeFromTrain = set(X_trainOhFeatures.columns) - sharedFeatures
    removeFromTest = set(X_testOhFeatures.columns) - sharedFeatures
    X_trainOhFeatures = X_trainOhFeatures.drop(list(removeFromTrain), axis=1)
    X_testOhFeatures = X_testOhFeatures.drop(list(removeFromTest), axis=1)

    # Apply logistic regression on the data, and calculate accuracy
    clf = svm.SVC(kernel='poly').fit(X_trainOhFeatures, y_train)
    predictions = clf.predict(X_testOhFeatures)
    accuracy = accuracy_score(y_test, predictions)*100
    print("Accuracy Score: ", accuracy)
    try:
      print('Precision Score : ', precision_score(y_test, predictions)*100)
      print('Recall Score : ', recall_score(y_test, predictions)*100)
      print('F1 Score : ', f1_score(y_test, predictions)*100)
      print('Confusion Matrix : \n' + str(confusion_matrix(y_test, predictions)))
    except:
      pass
    return (accuracy, )

In [ ]:
def getHof(toolbox):
    # Initialize variables to use eaSimple
    numPop = 100
    numGen = 50
    pop = toolbox.population(n=numPop)
    hof = tools.HallOfFame(numPop * numGen)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    # Launch genetic algorithm
    # change the crossover and mutation probability
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.65, mutpb=0.001, ngen=numGen, stats=stats, halloffame=hof, verbose=False)
    # Return the hall of fame
    return hof,log

In [ ]:
def getMetrics(hof, X_train, X_test, y_train, y_test):
    # Get list of percentiles in the hall of fame
    percentileList = [i / (len(hof) - 1) for i in range(len(hof))]
    # Gather fitness data from each percentile
    testAccuracyList = []
    validationAccuracyList = []
    individualList = []
    for individual in hof:
        testAccuracy = individual.fitness.values
        validationAccuracy = getFitness(individual, X_train, X_test, y_train, y_test)
        testAccuracyList.append(testAccuracy[0])
        validationAccuracyList.append(validationAccuracy[0])
        individualList.append(individual)
    #testAccuracyList.reverse()
    #validationAccuracyList.reverse()
    return testAccuracyList, validationAccuracyList, individualList, percentileList

In [ ]:
def drive(subject_name, channel_name):
  dfData = pd.read_csv('/content/drive/MyDrive/Sequential methods for channel selection/our code/New/' + subject_name + '_FourClass.csv')
  allFeatures = dfData
  names = allFeatures.columns
  scaler = MinMaxScaler()
  allFeatures = scaler.fit_transform(allFeatures)
  allFeatures = pd.DataFrame(allFeatures, columns=names)
  allFeatures = allFeatures.loc[:, allFeatures.apply(pd.Series.nunique) != 1]
  link = "/content/drive/MyDrive/Deap/" + subject_name + ".dat"
  with open(link, 'rb') as f:
    raw_data = pickle.load(f, encoding = 'latin1')
  labels = raw_data['labels']
  em_labels = emotion_label(labels, 'all') # get the emotion labels
  allClasses = np.array(em_labels)
  allFeatures, allClasses = shuffle(allFeatures, allClasses, random_state = 40)
  return allFeatures, allClasses

In [ ]:
subject_names = ["s01", "s02", "s03", "s04", "s05", "s06", "s07", "s08", "s09", "s10", "s11", "s12",
                 "s13", "s14", "s15", "s16", "s17", "s18", "s19", "s20", "s21",
                "s22", "s23", "s24", "s25", "s26", "s27", "s28", "s29", "s30", "s31", "s32"]
subject_all_channels = [['P7', 'P8', 'Fz', 'CP6', 'PO3', 'FC5', 'FC1'], ['AF3', 'CP1', 'F7', 'O1', 'AF4', 'FC6'],
                        ['O2', 'Oz', 'O1'], ['P8', 'CP2', 'Cz'], ['Fz', 'O2', 'PO4', 'FC1', 'F7', 'Oz', 'CP6', 'Fp2', 'FC6', 'C4', 'Pz', 'CP5', 'O1', 'T8'],
                        ['CP1', 'T7', 'FC6', 'AF3'], ['PO4', 'Pz', 'Fp2', 'Cz', 'P8', 'P7', 'O1', 'FC2', 'CP2'], ['Fp1', 'CP6', 'O2', 'C4', 'T8', 'AF3', 'PO3'],
                        ['T7', 'AF4', 'T8', 'P8', 'Cz'], ['T7', 'P3', 'FC1', 'CP1', 'Cz', 'PO3', 'CP2', 'O1'], ['C3', 'P7', 'FC2', 'F4', 'CP2', 'P3', 'F8', 'FC6', 'F7', 'FC5', 'FC1', 'F3', 'CP5', 'CP1'],
                        ['Cz', 'P3', 'F3', 'AF3'], ['Cz', 'AF3', 'Pz', 'Fp2', 'Oz', 'P4', 'Fz', 'FC2', 'AF4', 'P8', 'F8', 'P7', 'CP1'], ['FC2', 'PO3', 'P7'],
                        ['Pz', 'FC6', 'P4', 'Fp1', 'PO3', 'AF3'], ['O2', 'FC1', 'T8'], ['CP1', 'CP6', 'T8', 'AF4', 'PO4', 'O2', 'Pz', 'O1', 'F4', 'C4'],
                        ['FC6', 'AF4', 'Pz', 'F8', 'O1'], ['P8', 'CP6', 'T8', 'F4'], ['AF4', 'Fz', 'PO3', 'FC2', 'CP5', 'F8', 'P4', 'T7', 'F7', 'Fp2', 'FC5', 'PO4', 'C3'],
                        ['P7', 'T8', 'P4', 'Fz', 'F8', 'Fp2'], ['O2', 'C3', 'P4', 'PO4', 'F3', 'PO3', 'Fz', 'Fp2', 'AF4', 'T8', 'Fp1', 'CP1'],
                        ['T7', 'Fp1', 'CP5', 'P8', 'F7', 'F8', 'AF4', 'FC2'], ['Pz', 'P8', 'CP2', 'O1', 'P3', 'CP1', 'Cz', 'C3', 'FC2', 'T7', 'Oz', 'P7', 'AF4', 'Fp2', 'FC6', 'F3', 'P4', 'Fp1', 'C4', 'AF3', 'CP6'],
                        ['FC2', 'FC5', 'F4'], ['P3', 'FC5'], ['Oz', 'T8', 'AF4', 'FC5', 'CP6', 'FC6', 'CP5', 'O2', 'CP1', 'F7', 'C4', 'F3', 'CP2', 'C3', 'P4', 'F8', 'T7', 'Cz', 'Fz'],
                        ['CP2', 'C3', 'Oz', 'FC1', 'Pz', 'O2', 'PO4', 'P4', 'Fz', 'P8', 'CP5', 'PO3', 'Fp1', 'FC6', 'AF4', 'Fp2', 'C4', 'CP6'],
                        ['AF3', 'CP6', 'O2', 'F8', 'O1', 'CP1', 'CP5'], ['Oz', 'PO4'], ['C4', 'F4'], ['CP6', 'CP2', 'CP5', 'O1', 'PO4', 'Fz', 'AF4', 'Cz', 'Oz']]

In [ ]:
def main_code(allFeatures, allClasses):
  for i in range(0, 10):
    print("Fold value = ", i+1)
    print("-"*40)
    #call k fold
    X_train, X_test, y_train, y_test = kfold(allFeatures, allClasses, i)
    #==========================    DEAP GLOBAL VARIABLES (viewable by SCOOP)      ======================
    # Create Individual
    creator.create("FitnessMax", base.Fitness, weights = (1.0,))
    creator.create("Individual", list, fitness = creator.FitnessMax)

    # Create Toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, len(allFeatures.columns) - 1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Continue filling toolbox...
    toolbox.register("evaluate", getFitness, X_train = X_train, X_test = X_test, y_train = y_train, y_test = y_test)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb = 0.05)
    toolbox.register("select", tools.selTournament, tournsize = 7)
    #===================================================================================================
    
    # First, we will apply logistic regression using all the features to acquire a baseline accuracy.
    individual = [1 for i in range(len(allFeatures.columns))]
    Accuracy = getFitness_all(individual, X_train, X_test, y_train, y_test)
    if(Accuracy[0]==100):
      continue
    #print('\nAccuracy with all features from SVM: \t', Accuracy)
    # Now, we will apply a genetic algorithm to choose a subset of features that gives a better accuracy than the baseline.
    hof,log = getHof(toolbox)
    testAccuracyList, validationAccuracyList, individualList, percentileList = getMetrics(hof, X_train, X_test, y_train, y_test)
    # Get a list of subsets that performed best on validation data
    maxValAccSubsetIndicies = [index for index in range(len(validationAccuracyList)) if validationAccuracyList[index] == max(validationAccuracyList)]
    maxValIndividuals = [individualList[index] for index in maxValAccSubsetIndicies]
    maxValSubsets = [[list(allFeatures)[index] for index in range(len(individual)) if individual[index] == 1] for individual in maxValIndividuals]
    acc_list, predict_list = [], []
    print('Optimal Feature Subset(s):')
    for i in range(len(maxValAccSubsetIndicies)):
        x_train = X_train[maxValSubsets[i]]
        x_test = X_test[maxValSubsets[i]]
        try:
          clf = svm.SVC(kernel = 'poly').fit(x_train, y_train)
          y_pred_val = clf.predict(x_test)
          acc_list.append(accuracy_score(y_test, y_pred_val)*100)
          predict_list.append(y_pred_val)
        except:
          pass
          # store the accuracy, precission and recall in a list then for each fold
    # convert list to array
    acc_list = np.array(acc_list)
    # find the maximum accuracy
    result = np.where(acc_list == np.amax(acc_list))[0] # now result has all the highest accuracy index
    # check for the precission
    temp_acc = acc_list[result[0]]
    print("Accurcay after fs: ", temp_acc)
    draftlist = np.array(predict_list[result[0]])
    print(classification_report(y_test, draftlist))
    print("-"*40)

In [ ]:
for i in range(27, 32):
  print('-'*100)
  print("Subject No: ", subject_names[i])
  print('-'*100)
  allFeatures, allClasses = drive(subject_names[i], subject_all_channels[i])
  main_code(allFeatures, allClasses)

# Result Analysis

In [ ]:
[79.6, 61.1, 80.5, 67.8, 66.2, 71.7, 67.6, 68.4, 62.5, 79.2, 48.7, 91.2, 76.3, 72, 75.4, 79.8, 63.1, 67.1, 76.2, 66.3, 62.3, 62.4, 82.8, 49.3, 81.6, 89.4, 60.6, 77.7, 71.3, 74.6, 65.1, 75.6]

In [ ]:
[80, 62.5, 77.5, 72.5, 62.5, 72.5, 65, 70, 65, 80, 53.3, 92.5, 77.5, 77.5, 75, 80, 65, 74.2, 76.7, 70, 65.8, 60, 85, 55, 83.3, 90, 68.3, 77.5, 75, 75, 67.5, 75]

In [2]:
import numpy as np
accuracy = np.array([77.5, 62.5, 77.5, 72.5, 62.5, 72.5, 65, 67.5, 65, 80, 52.5, 92.5, 77.5, 77.5, 75, 80, 57.5, 72.5, 75, 
                     70, 65, 60, 85, 55, 82.5, 90, 67.5, 77.5, 75, 75, 67.5, 75])
print(accuracy.shape)
print(np.mean(accuracy))
print(np.std(accuracy))

(32,)
72.109375
9.293591857262454


In [3]:
pre = np.array([84, 66.6, 89.4, 66.7, 79.5, 76.8, 81, 73.7, 66.3, 82.5, 48.8, 91.4, 79.4, 69.7, 78.7, 84.6, 72.6, 
                64.1, 83.4, 67.1, 66.9, 70.7, 83.8, 53.6, 84.5, 90.8, 58, 83.3, 70.3, 79.8, 68.2, 79.9])
print(pre.shape)
print(np.mean(pre))
print(np.std(pre))

(32,)
74.878125
10.337061187028691


In [4]:
recall = np.array([80, 62.5, 77.5, 72.5, 62.5, 72.5, 65, 70, 65, 80, 53.3, 92.5, 77.5, 77.5, 75, 80, 65, 74.2, 76.7, 70, 65.8, 60, 85,
                   55, 83.3, 90, 68.3, 77.5, 75, 75, 67.5, 75])
print(recall.shape)
print(np.mean(recall))
print(np.std(recall))

(32,)
72.70625
8.999963107563275


In [5]:
f1 = np.array([79.6, 61.1, 80.5, 67.8, 66.2, 71.7, 67.6, 68.4, 62.5, 79.2, 48.7, 91.2, 76.3, 72, 75.4, 79.8, 63.1, 67.1, 76.2, 
               66.3, 62.3, 62.4, 82.8, 49.3, 81.6, 89.4, 60.6, 77.7, 71.3, 74.6, 65.1, 75.6])
print(f1.shape)
print(np.mean(f1))
print(np.std(f1))

(32,)
71.04374999999999
9.753971546887966
